<a href="https://colab.research.google.com/github/componavt/sns4human/blob/main/src/vk/nlp/get_posts_tokens_with_group_id.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script processes text data, tokenizes it and saves it to a CSV file, taking into account the posts' affiliation to a specific VK group.

Этот скрипт обрабатывает текстовые данные, токенизирует их и сохраняет в CSV-файл с учетом принадлежности постов к конкретной ВК группе.

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
!pip install -U pymorphy3
import pymorphy3
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
from nltk.corpus import stopwords
stop_words = stopwords.words("russian")
alphabet = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя-')
morph = pymorphy3.MorphAnalyzer(lang='ru')

In [ ]:
domains = ['satasanaa', 'speechvepkar', 'desyatiletieyazykovkarelia', 'learning_karelian_language']

temp_df = []
for domain in domains:
    t = pd.read_csv(f'https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/data/vk/posts/{domain}.csv', usecols=['text', 'date'])
    t['group_name'] = domain
    temp_df.append(t)

df = pd.concat(temp_df, ignore_index=True).sort_values('date')
df = df[df['text'].notna() & (df['text'].apply(lambda x: isinstance(x, str))) & (df['text'] != '') ]

In [ ]:
def process_text(text):
    check_hash = False
    processed_parts = []
    for w in nltk.word_tokenize(text):
      if len(w) == 1:
        continue
      if w == '#':
          check_hash = True
          continue
      if check_hash:
          check_hash = False
          continue
      w_tag = morph.parse(w.strip())[0].tag
      if   'Surn' in w_tag or 'Name' in w_tag or 'Patr' in w_tag:
         continue
      if set(w.lower()).issubset(alphabet):
        if w.isalpha() and w.lower():
          if w.isupper() and len(w) <= 3:
              processed_parts.append(w)
          else:
              res = morph.parse(w.lower())[0].normal_form
              if res not in stop_words:
                  processed_parts.append(res)
    result = ' '.join(processed_parts)
    return str(result)

df['tokens'] = df['text'].apply(lambda x: process_text(x))
df = df[df['tokens'].notna() & (df['tokens'].apply(lambda x: isinstance(x, str))) & (df['tokens'] != '') & (df['tokens'] != ' ')]
df_tokens = pd.concat([df['tokens'], df['date'],df['group_name']], axis=1, keys=['tokens', 'date','group_name'])

df_tokens.to_csv('tokens.csv', index=False)